In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Cargar los datos de los portafolios generados por el backtesting
portfolios_df = pd.read_csv('reports/strategies_portfolio.csv', parse_dates=['date'])

# --- Prueba de Hipótesis: t-test con corrección Newey-West ---

# 1. Calcular los retornos diarios para cada estrategia
returns_df = portfolios_df.set_index('date').pct_change().dropna()

# 2. Crear las series de diferencias de retornos
# H1: Walk-Forward es mejor que Buy & Hold
returns_df['diff_wf_vs_bh'] = returns_df['walk_forward_portfolio'] - returns_df['buy_and_hold_portfolio']
# H1: Walk-Forward es mejor que DCA
returns_df['diff_wf_vs_dca'] = returns_df['walk_forward_portfolio'] - returns_df['dca_portfolio']

def perform_newey_west_ttest(diff_series, strategy_name):
    """
    Realiza un t-test sobre la media de una serie de diferencias de retornos,
    utilizando errores estándar robustos a heterocedasticidad y autocorrelación (Newey-West).
    
    La hipótesis nula (H0) es que la media de las diferencias es cero.
    La hipótesis alternativa (H1) es que la media de las diferencias es mayor que cero.
    """
    
    # Preparamos los datos para el modelo de regresión (regresión contra una constante)
    y = diff_series.dropna()
    X = sm.add_constant(np.ones(len(y))) # Vector de unos para estimar el intercepto (la media)
    
    # Estimamos el lag óptimo para la corrección de Newey-West (regla de pulgar)
    max_lags = int(np.floor(len(y)**(1/4)))
    
    # Ajustamos el modelo OLS con la corrección de covarianza
    model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': max_lags})
    
    # Extraer resultados
    mean_diff = model.params[0]
    t_stat = model.tvalues[0]
    p_value = model.pvalues[0] / 2  # Dividimos por 2 para un test de una cola (H1: media > 0)
    
    print(f"--- Comparación: Walk-Forward vs. {strategy_name} ---")
    print(f"Diferencia Media en Retornos Diarios: {mean_diff:.6f}")
    print(f"Estadístico t (Newey-West): {t_stat:.4f}")
    print(f"P-valor (una cola): {p_value:.6f}")
    
    # Conclusión
    alpha = 0.05
    if p_value < alpha and mean_diff > 0:
        print(f"Conclusión: Se rechaza la hipótesis nula (H0) con un nivel de significancia del {alpha*100}%.")
        print("La estrategia Walk-Forward es estadísticamente superior.\n")
    else:
        print(f"Conclusión: No se puede rechazar la hipótesis nula (H0).")
        print("No hay evidencia estadística suficiente para afirmar que Walk-Forward es superior.\n")

# Ejecutar las pruebas
perform_newey_west_ttest(returns_df['diff_wf_vs_bh'], "Buy & Hold")
perform_newey_west_ttest(returns_df['diff_wf_vs_dca'], "Dollar Cost Averaging (DCA)")